# Nashville Polive Service Calls Analysis

## Dependencies

In [45]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import requests
import sodapy

ModuleNotFoundError: No module named 'sodapy'

In [47]:
!conda install sodapy -y

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - sodapy

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



PackagesNotFoundError: The following packages are not available from current channels:

  - sodapy

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate ch

### Import

In [54]:
url = 'https://data.nashville.gov/resource/kwnd-qrrm.json'

In [55]:
r = requests.get(url)

In [56]:
json = r.json()

In [59]:
test_df = pd.read_json(url)

In [60]:
test_df.shape

(1000, 23)

In [44]:
test_df.columns

Index(['event_number', 'call_rec', 'tencode', 'description', 'tencode_suf',
       'unit_dispatched', 'shift', 'sector', 'zone', 'rpa',
       'tencode_suf_description', 'disposition_code',
       'disposition_description', 'block', 'street_name', 'lat', 'lon',
       'mapped_location', ':@computed_region_wvby_4s8j',
       ':@computed_region_3aw5_2wv7', ':@computed_region_p6sk_2acq',
       ':@computed_region_gxvr_9jxz', 'complaint'],
      dtype='object')

In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.nashville.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.nashville.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("kwnd-qrrm", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [3]:
main_df = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv')

<ipython-input-3-572f7c7c68b9>:1: DtypeWarning: Columns (10,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv')


In [ ]:
main_df.to_feather('data/calls.feather')

### Preprocessing

* Let's get a sample of the data to see what we're working with.

In [15]:
samp_df = main_df.sample(frac=.01, random_state=22)

#### 'Event Number'

* It looks like all the event numbers begin with 'PD'; if this is the case, then I can strip those two characters and cast as an int, saving space

In [20]:
pd_check = [event.startswith('PD') for event in samp_df['Event Number'].values]

In [23]:
print(sum(pd_check))

65701


In [24]:
def event_number_clean(num):
    return int(num[2:])

In [25]:
samp_df['Event Number'] = samp_df['Event Number'].apply(event_number_clean)

In [34]:
samp_df.iloc[[0]]

,Event Number,Call Received,Complaint Number,Tencode,Tencode Description,Tencode Suffix,Tencode Suffix Description,Disposition Code,Disposition Description,Block,Street Name,Unit Dispatched,Shift,Sector,Zone,RPA,Latitude,Longitude,Mapped Location
5797724,201601214292,12/01/2016 04:51:05 PM,NaN,88,INVESTIGATE 911 HANG-UP CALL,PW,NaN,5,GONE ON ARRIVAL,NaN,NaN,322B,B,S,327.0,8861.0,NaN,NaN,NaN


## EDA

In [5]:
main_df.columns

Index(['Event Number', 'Call Received', 'Complaint Number', 'Tencode',
       'Tencode Description', 'Tencode Suffix', 'Tencode Suffix Description',
       'Disposition Code', 'Disposition Description', 'Block', 'Street Name',
       'Unit Dispatched', 'Shift', 'Sector', 'Zone', 'RPA', 'Latitude',
       'Longitude', 'Mapped Location'],
      dtype='object')

In [7]:
main_df.dtypes

Event Number                   object
Call Received                  object
Complaint Number              float64
Tencode                         int64
Tencode Description            object
Tencode Suffix                 object
Tencode Suffix Description     object
Disposition Code               object
Disposition Description        object
Block                         float64
Street Name                    object
Unit Dispatched                object
Shift                          object
Sector                         object
Zone                           object
RPA                           float64
Latitude                      float64
Longitude                     float64
Mapped Location                object
dtype: object

In [12]:
cols = ['Event Number', 'Call Received', 'Complaint Number', 'Tencode',
       'Tencode Description', 'Tencode Suffix', 'Tencode Suffix Description',
       'Disposition Code', 'Disposition Description', 'Block', 'Street Name',
       'Unit Dispatched', 'Shift', 'Sector', 'Zone', 'RPA', 'Latitude',
       'Longitude', 'Mapped Location']

for col in cols:
    print(f'Column name: {col}')
    print(main_df[col].head(10))
    print('\n*******\n')

Column name: Event Number
0    PD202100795991
1    PD201900309484
2    PD201500826444
3    PD202000693991
4    PD201501158284
5    PD201500812635
6    PD201500425464
7    PD201900614872
8    PD201700837924
9    PD201900452295
Name: Event Number, dtype: object

*******

Column name: Call Received
0    12/25/2021 02:40:56 PM
1    04/08/2019 09:39:17 AM
2    08/04/2015 02:39:43 PM
3    09/16/2020 10:29:30 AM
4    11/05/2015 06:30:42 AM
5    07/31/2015 09:34:06 AM
6    04/23/2015 07:57:37 AM
7    07/15/2019 09:42:20 AM
8    08/17/2017 12:56:36 PM
9    05/23/2019 12:49:06 AM
Name: Call Received, dtype: object

*******

Column name: Complaint Number
0    2.021065e+10
1    2.019027e+10
2             NaN
3    2.020060e+10
4             NaN
5             NaN
6             NaN
7    2.019053e+10
8    2.017073e+10
9             NaN
Name: Complaint Number, dtype: float64

*******

Column name: Tencode
0    50
1    50
2    75
3    50
4    93
5    93
6    93
7    50
8    50
9    93
Name: Tencode, dty